In [1]:
import os
import sys
import glob
import datetime

import numpy as np
import pandas as pd

In [43]:
gp_table = pd.read_table('crab_gp_kaz_10_2010-2018.csv', sep='\t')
gp_table['Time_series'] = pd.to_datetime(gp_table['Date'], format='%d.%m.%Y', errors='ignore')
gp_table = gp_table.set_index('Time_series')
print(len(gp_table))

1378


## Удаление аномальных событий и калибровочных ступений

In [44]:
no_GP = pd.read_table('noGI.csv', sep=',')

In [45]:
no_GP

,дата,номер импульса,Примечание
0,2010-10-23,0,anomal wide
1,2010-10-23,1,calib
2,2010-10-23,2,calib
3,2010-10-23,3,calib
4,2010-10-23,4,calib
5,2010-10-23,5,calib
6,2010-10-23,6,calib
7,2010-10-23,7,calib
8,2010-10-23,8,calib
9,2010-10-23,9,calib


In [46]:
gp_table.head()

,Date,Time start,"Tay, ms","Period, s","Numpointwin, point","Numpulse, a.u.","Median, adc u","STD, adc u",path obs plot,path obs data,"Count of GP, u","point of gp, point","amp of gp, adc u","W50, point","W10, point",path plot
Time_series,,,,,,,,,,,,,,,,
2010-09-09,09.09.2010,06:48:41.8374290,2.4576,5.988578,2430,34,479.795355,12.035700,./obs_plot/09.09.2010_plot_0531+21.png,.\obs_data\09.09.2010_obs_0531+21.csv,1,45640,600.19722,99.00000,99.0,./gp_plot/09.09.2010_plot_0531+21_0.png
2010-09-12,12.09.2010,06:36:54.1340866,2.4576,5.988594,2430,34,466.859325,10.627263,./obs_plot/12.09.2010_plot_0531+21.png,.\obs_data\12.09.2010_obs_0531+21.csv,1,36629,580.26615,99.00000,99.0,./gp_plot/12.09.2010_plot_0531+21_0.png
2010-10-17,17.10.2010,04:19:17.6313152,2.4576,5.988894,2430,34,651.843725,15.661569,./obs_plot/17.10.2010_plot_0531+21.png,.\obs_data\17.10.2010_obs_0531+21.csv,1,27361,883.96305,99.00000,99.0,./gp_plot/17.10.2010_plot_0531+21_0.png
2010-10-23,23.10.2010,03:55:42.2248611,2.4576,5.988966,2430,34,246.820140,21.169761,./obs_plot/23.10.2010_plot_0531+21.png,.\obs_data\23.10.2010_obs_0531+21.csv,1,74376,807.85562,73.67085,99.0,./gp_plot/23.10.2010_plot_0531+21_0.png
2010-10-23,23.10.2010,03:55:42.2248611,2.4576,5.988966,2430,34,246.820140,21.169761,./obs_plot/23.10.2010_plot_0531+21.png,.\obs_data\23.10.2010_obs_0531+21.csv,1,74359,779.00989,16.65180,99.0,./gp_plot/23.10.2010_plot_0531+21_1.png


In [47]:
gp_table[(gp_table.index == '2010-10-17')]

,Date,Time start,"Tay, ms","Period, s","Numpointwin, point","Numpulse, a.u.","Median, adc u","STD, adc u",path obs plot,path obs data,"Count of GP, u","point of gp, point","amp of gp, adc u","W50, point","W10, point",path plot
Time_series,,,,,,,,,,,,,,,,
2010-10-17,17.10.2010,04:19:17.6313152,2.4576,5.988894,2430,34,651.843725,15.661569,./obs_plot/17.10.2010_plot_0531+21.png,.\obs_data\17.10.2010_obs_0531+21.csv,1,27361,883.96305,99.0,99.0,./gp_plot/17.10.2010_plot_0531+21_0.png


In [48]:
for index, row in no_GP.iterrows():
    print(row['дата'], row['номер импульса'])
    try:
        gp_table = gp_table.drop(
            gp_table[
                ((gp_table.index == row['дата']) 
                 & (gp_table['path plot'].str.contains('_' + str(row['номер импульса']) + '.png')))].index)
    except TypeError:
        continue

2010-10-23 0
2010-10-23 1
2010-10-23 2
2010-10-23 3
2010-10-23 4
2010-10-23 5
2010-10-23 6
2010-10-23 7
2010-10-23 8
2010-10-23 9
2010-10-23 10
2010-10-23 11
2010-11-25 0
2010-11-25 1
2011-04-20 0
2011-04-20 1
2011-04-20 2
2011-04-20 3
2011-04-20 4
2011-04-20 5
2011-04-20 6
2011-04-20 7
2011-04-20 8
2011-04-20 9
2011-04-20 10
2011-04-20 11
2011-04-20 12
2011-04-20 13
2011-04-20 14
2011-04-20 15
2011-04-20 16
2011-05-05 0
2011-05-17 0
2011-05-17 1
2011-05-17 2
2011-05-17 3
2011-05-17 4
2011-06-14 0
2011-06-14 1
2011-06-16 0
2011-06-20 0
2011-06-20 1
2011-06-20 2
2011-06-20 3
2011-06-20 4
2011-06-20 5
2011-06-20 6
2011-06-20 7
2011-06-20 8
2011-06-20 9
2011-06-20 10
2011-06-20 11
2011-06-20 12
2011-06-20 13
2011-06-20 14
2011-06-20 15
2011-06-20 16
2011-06-20 17
2011-06-20 18
2011-06-20 19
2011-06-20 20
2011-06-20 21
2011-06-20 22
2011-06-20 23
2011-06-20 24
2011-06-20 25
2011-06-20 26
2011-06-20 27
2011-06-20 28
2011-06-20 29
2011-06-20 30
2011-06-20 31
2011-06-20 32
2011-06-20 33
2011-

<generator object DataFrame.iterrows at 0x0000022FAEC4C678>


In [79]:
type(gp_table)

pandas.core.frame.DataFrame

In [49]:
gp_table

,Date,Time start,"Tay, ms","Period, s","Numpointwin, point","Numpulse, a.u.","Median, adc u","STD, adc u",path obs plot,path obs data,"Count of GP, u","point of gp, point","amp of gp, adc u","W50, point","W10, point",path plot
Time_series,,,,,,,,,,,,,,,,
2010-09-09,09.09.2010,06:48:41.8374290,2.4576,5.988578,2430,34,479.795355,12.035700,./obs_plot/09.09.2010_plot_0531+21.png,.\obs_data\09.09.2010_obs_0531+21.csv,1,45640,600.19722,99.00000,99.0,./gp_plot/09.09.2010_plot_0531+21_0.png
2010-09-12,12.09.2010,06:36:54.1340866,2.4576,5.988594,2430,34,466.859325,10.627263,./obs_plot/12.09.2010_plot_0531+21.png,.\obs_data\12.09.2010_obs_0531+21.csv,1,36629,580.26615,99.00000,99.0,./gp_plot/12.09.2010_plot_0531+21_0.png
2010-10-17,17.10.2010,04:19:17.6313152,2.4576,5.988894,2430,34,651.843725,15.661569,./obs_plot/17.10.2010_plot_0531+21.png,.\obs_data\17.10.2010_obs_0531+21.csv,1,27361,883.96305,99.00000,99.0,./gp_plot/17.10.2010_plot_0531+21_0.png
2010-10-27,27.10.2010,03:39:58.6513328,2.4576,5.989016,2430,34,529.343655,12.211583,./obs_plot/27.10.2010_plot_0531+21.png,.\obs_data\27.10.2010_obs_0531+21.csv,1,53762,711.86288,99.00000,99.0,./gp_plot/27.10.2010_plot_0531+21_0.png
2010-10-28,28.10.2010,03:36:02.7297437,2.4576,5.989029,2430,34,680.000655,16.059009,./obs_plot/28.10.2010_plot_0531+21.png,.\obs_data\28.10.2010_obs_0531+21.csv,1,10499,1052.06685,99.00000,99.0,./gp_plot/28.10.2010_plot_0531+21_0.png
2010-11-09,09.11.2010,02:48:51.9448242,2.4576,5.989196,2430,34,785.025420,19.094841,./obs_plot/09.11.2010_plot_0531+21.png,.\obs_data\09.11.2010_obs_0531+21.csv,1,859,1057.49185,99.00000,99.0,./gp_plot/09.11.2010_plot_0531+21_0.png
2010-11-11,11.11.2010,02:41:00.1358726,2.4576,5.989226,2430,34,710.573110,15.706577,./obs_plot/11.11.2010_plot_0531+21.png,.\obs_data\11.11.2010_obs_0531+21.csv,1,59983,909.40773,99.00000,99.0,./gp_plot/11.11.2010_plot_0531+21_0.png
2010-11-19,19.11.2010,02:09:32.9584922,2.4576,5.989348,2430,34,523.813060,42.777546,./obs_plot/19.11.2010_plot_0531+21.png,.\obs_data\19.11.2010_obs_0531+21.csv,1,35028,1062.29514,98.53965,99.0,./gp_plot/19.11.2010_plot_0531+21_0.png
2010-12-12,12.12.2010,00:39:07.2615385,2.4576,5.989727,2430,34,554.131990,12.696776,./obs_plot/12.12.2010_plot_0531+21.png,.\obs_data\12.12.2010_obs_0531+21.csv,1,63425,688.23183,99.00000,99.0,./gp_plot/12.12.2010_plot_0531+21_0.png


In [50]:
gp_table.to_csv('crab_gp_kaz_10_2016-2018.csv',  sep='\t', header=True, index=False)

In [83]:
row['номер импульса']

0

In [98]:
gp_table.drop(
    gp_table[
        (gp_table.index == '2017-02-03')
        & (gp_table['path plot'].str.contains('_1.png'))].index)

TypeError: 'NoneType' object is not iterable

In [35]:
df = pd.DataFrame(np.arange(24).reshape(6,4),columns=['A', 'B', 'C', 'D'])

In [40]:
df

,A,B,C,D
0,0,1,2,3
1,4,5,6,ada
2,8,9,10,bada
3,12,13,14,15
4,16,17,18,19
5,20,21,22,23


In [37]:
dd = pd.DataFrame([[1, 'ada'],[2, 'bada'],[5, '3']],columns=['A1', 'B1'])

In [32]:
dd

,A1,B1
0,1,ada
1,2,bada
2,5,3


In [41]:
for index, row in dd.iterrows():
    df = df.drop(df[(df.index == row['A1']) & (df['D'].str.contains(row['B1']))].index)

In [42]:
df

,A,B,C,D
0,0,1,2,3
3,12,13,14,15
4,16,17,18,19
5,20,21,22,23
